In [1]:
import geopandas as gpd
import contextily as ctx
import skmob
from tqdm import tqdm
import pandas as pd
from datetime import datetime

## REGIONS

In [2]:
city_regions = gpd.read_file('../data/raw/KBR/Rejony/EtapII-REJONY_wroclaw.shp')

In [3]:
city_regions.head()

,NUMBER,NAME,ZST_0_5,ZST_6_15,ZST_16_19,ZST_20_24,ZST_25_44,ZST_45_WE,ZST_WE_I_W,ZST_SUMA,...,ZCZ_25_44,ZCZ_45_WE,ZCZ_WE_I_W,ZCZ_SUMA,GUS_MI,GUS_MI_6_,BD_A_MI_6_,REGON_ZI,BD_A_ZI,geometry
0,22,Komandorska/Swobodna,111.0,200.0,68.0,82.0,670.0,606.0,1412.0,3149.0,...,32.0,11.0,3.0,64.0,3213,3102,3202,1255,1200,"POLYGON ((6432254.582 5662772.742, 6432255.097..."
1,23,Centrum PoÅudniowe,101.0,124.0,49.0,81.0,595.0,547.0,812.0,2309.0,...,55.0,17.0,3.0,95.0,2404,2303,2377,10258,4750,"POLYGON ((6432010.645 5662990.713, 6431954.553..."
2,24,Stysia,154.0,257.0,74.0,123.0,942.0,825.0,1257.0,3632.0,...,34.0,13.0,4.0,137.0,3769,3615,5118,1373,2296,"POLYGON ((6431698.436 5663205.105, 6431588.369..."
3,25,OstrÃ³w Tumski,20.0,26.0,10.0,18.0,219.0,220.0,227.0,740.0,...,22.0,10.0,14.0,54.0,794,774,799,413,1479,"POLYGON ((6433350.626 5664536.371, 6433232.522..."
4,26,Szczytnicka,221.0,320.0,139.0,274.0,1375.0,1349.0,1134.0,4812.0,...,49.0,13.0,3.0,94.0,4906,4685,4836,1468,1522,"POLYGON ((6433801.565 5665020.377, 6433884.529..."


In [4]:
city_regions = city_regions[['NUMBER', 'geometry']]
city_regions.columns = ['REGION', 'geometry']
city_regions = city_regions.to_crs(epsg=3857)

In [5]:
# ax = city_regions.plot(figsize=(10,10), alpha=0.5, edgecolor='k')
# ctx.add_basemap(ax)
# ax.set_axis_off()

## INTERREGIONAL TRAVELS

In [6]:
interregional_travels = gpd.read_file('../data/raw/KBR/SIM/EtapV-11_Wiezba_ruchu_SIM_podroze-miedzyrejonowe.shp')

In [7]:
interregional_travels.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 225625 entries, 0 to 225624
Data columns (total 30 columns):
 #   Column      Non-Null Count   Dtype   
---  ------      --------------   -----   
 0   GROUP       225625 non-null  object  
 1   REJ_ZRODLO  225625 non-null  int64   
 2   REJ_CEL     225625 non-null  int64   
 3   RELACJA     225625 non-null  object  
 4   H_0         225625 non-null  int64   
 5   H_1         225625 non-null  int64   
 6   H_2         225625 non-null  int64   
 7   H_3         225625 non-null  int64   
 8   H_4         225625 non-null  int64   
 9   H_5         225625 non-null  int64   
 10  H_6         225625 non-null  int64   
 11  H_7         225625 non-null  int64   
 12  H_8         225625 non-null  int64   
 13  H_9         225625 non-null  int64   
 14  H_10        225625 non-null  int64   
 15  H_11        225625 non-null  int64   
 16  H_12        225625 non-null  int64   
 17  H_13        225625 non-null  int64   
 18  H_14        2256

In [8]:
interregional_travels.head()

,GROUP,REJ_ZRODLO,REJ_CEL,RELACJA,H_0,H_1,H_2,H_3,H_4,H_5,...,H_16,H_17,H_18,H_19,H_20,H_21,H_22,H_23,DOBA,geometry
0,1,1,1,"1,1",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"LINESTRING (6432281.815 5664267.514, 6432281.8..."
1,2,1,2,"1,2",9,9,17,9,0,0,...,52,87,44,61,35,26,26,17,1035,"LINESTRING (6432281.815 5664267.514, 6432379.5..."
2,3,1,3,"1,3",52,0,0,9,9,0,...,87,113,61,9,44,26,26,17,1253,"LINESTRING (6432281.815 5664267.514, 6432781.8..."
3,4,1,4,"1,4",0,0,0,0,0,0,...,70,87,44,52,35,0,9,9,861,"LINESTRING (6432281.815 5664267.514, 6432720.5..."
4,5,1,5,"1,5",0,0,0,0,0,9,...,87,78,17,26,44,9,26,0,1288,"LINESTRING (6432281.815 5664267.514, 6432576.1..."


In [9]:
interregional_travels.rename(
    columns={
        'REJ_ZRODLO': 'ORIGIN',
        'REJ_CEL': 'DESTINATION'
    },
    inplace=True
)

In [10]:
interregional_travels = interregional_travels.drop(columns=['geometry'])

In [11]:
interregional_travels = interregional_travels[interregional_travels['ORIGIN'].isin(city_regions['REGION'])]
interregional_travels = interregional_travels[interregional_travels['DESTINATION'].isin(city_regions['REGION'])]

In [12]:
travels = pd.DataFrame(columns=['HOUR', 'ORIGIN', 'DESTINATION', 'FLOW'])

for i in tqdm(range(24)):
    sub_df = interregional_travels[['DESTINATION', 'ORIGIN', f'H_{i}']]
    sub_df.rename(
        columns={
            f'H_{i}': 'FLOW'
        },
        inplace=True
    )
    # sub_df['HOUR'] = f'{str(i)}:00:00'
    sub_df['HOUR'] = datetime(2020, 1, 1, i, 1, 0)
    travels = travels.append(sub_df, ignore_index=True)

100%|██████████| 24/24 [00:01<00:00, 14.30it/s]


## FLOW DATAFRAME

In [13]:
city_regions = city_regions.to_crs(epsg=4326)

In [14]:
travels_from = travels[travels['ORIGIN'] == 156]
travels_to = travels[travels['DESTINATION'] == 156]

In [15]:
fdf_from = skmob.FlowDataFrame(
    data=travels_from,
    origin='ORIGIN',
    destination='DESTINATION',
    flow='FLOW',
    tile_id='REGION',
    tessellation=city_regions,
    datetime='HOUR',
    timestamp=True
)

In [16]:
fdf_to = skmob.FlowDataFrame(
    data=travels_to,
    origin='ORIGIN',
    destination='DESTINATION',
    flow='FLOW',
    tile_id='REGION',
    tessellation=city_regions,
    datetime='HOUR',
    timestamp=True
)

In [17]:
# m = fdf_from.plot_tessellation(tiles='OpenStreetMap')
# fdf_from.plot_flows(m, flow_color='red', tiles='OpenStreetMap', min_flow=10)
# fdf_to.plot_flows(m, flow_color='blue', tiles='OpenStreetMap', min_flow=10)

In [18]:
# m = fdf_from.plot_tessellation(tiles='OpenStreetMap')
# fdf_from.plot_flows(m, flow_color='red', tiles='OpenStreetMap', min_flow=10, slider=True)
# fdf_to.plot_flows(m, flow_color='blue', tiles='OpenStreetMap', min_flow=10, slider=True)

In [19]:
fdf_from.plot_flows(flow_color='red', tiles='OpenStreetMap', min_flow=1)

In [20]:
fdf_to.plot_flows(flow_color='blue', tiles='OpenStreetMap', min_flow=10)